In [ ]:
import pandas as pd
import glob
import numpy as np
import os 
import sys


In [ ]:
sheets = np.array(glob.glob( "S*.xlsx"))
print(sheets)

In [ ]:
#Remove min, max, stdv, range columns that don't serve ML purposes 
phrases = [   "Unnamed" , "SASA" , "5.0" , "range" , "volume" , "pyramidalization" , "4.0" ]
for i in range (len(sheets)):
    df = pd.read_excel(sheets[i])
    elimCol = [col for col in df.columns if any(phrase in col for phrase in phrases)]
    df_filtered = df.drop(columns=elimCol)
    allColumns = list(df_filtered.columns)
    sterCol = [col for col in allColumns if "Sterimol" in col and "Buried" not in col]

    dfNew = df_filtered.drop(columns = sterCol)
    saveStr = "Properties_MAST_" + str(i) + ".xlsx"

    dfNew.to_excel(saveStr, index=False)

In [ ]:
sheets2 = np.array(glob.glob( "P*.xlsx"))


In [ ]:
allLabels = []
for i in range (len(sheets2)):
    print(i)
    df = pd.read_excel(sheets2[i])
    print(df.shape)
    column_labels = df.columns.values
    allLabels.append(column_labels)
#Take all column headers for the new Summary_Properties_MAST into an array allLabels

In [ ]:
#Remove min, max, stdv, range columns that don't serve ML purposes 
phrases = [   "Unnamed"  ]
for i in range (len(sheets)):
    df = pd.read_excel(sheets[i])
    columns_to_remove = [col for col in df.columns if any(phrase in col for phrase in phrases)]
    df_filtered = df.drop(columns=columns_to_remove)

    saveStr = "Properties_MAST_" + str(i) + ".xlsx"

    df_filtered.to_excel(saveStr, index=False)

In [ ]:
common = set(allLabels[0])
for array in allLabels[1:]:
    common.intersection_update(array)
commonLabelsMAST = list(common)
#Find intersections using python's integrated set (works through hashing and iteration)

In [ ]:
for i in range (len(sheets2)):
    print(sheets2[i])
    if i == 0:
        df = pd.read_excel(sheets2[i])
        #print(df)
        df_MAST = df[commonLabelsMAST]
        #print(df_MAST)
    else:
        df = pd.read_excel(sheets2[i])
        rel_df = df[commonLabelsMAST]
        
        df_MAST = pd.concat([df_MAST, rel_df], axis=0)
    print(df.shape)

    print(df_MAST)
#Take from the new Summary_Properties_MAST, only the data present in the columns of interest and concatenate 

In [ ]:
df_sorted = df_MAST[sorted(df_MAST.columns)]

print(df_sorted)
#Revert to Smiles Structure 
df_sorted = df_sorted[["Compound_Name"] + [col for col in df_sorted.columns if col != "Compound_Name"]]

df_sorted.to_excel("sudipALKS_SummaryProps.xlsx", index=False)

In [ ]:
#Load MASTER File 
df_MASTER = pd.read_excel("mCPBAALKS_SummaryProps.xlsx")

smilesDir = "/Users/danielruiz/Downloads/mCPBA_Lit_Database_01_27_25/MasterDataFramemCPBA.csv"

nameDir = "/Users/danielruiz/Downloads/mCPBA_Lit_Database_01_27_25/features/DFTInput/masterDF_KMeansClustered_mCPBA_rdkitmorganDFTInput.csv"


In [ ]:
def convertCanonical(str):
    from rdkit import Chem
    print(str)
    mol = Chem.MolFromSmiles(str)
    canonical = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    return canonical

In [ ]:

nameDF = pd.read_csv(nameDir)
smilesDF = pd.read_csv(smilesDir)
smilesDF.insert(5, "Canonical", 0)
smiles_ =  list(smilesDF["SMILES"])
for i , smile in enumerate(smiles_):
    canonical = convertCanonical(smile)
    smilesDF["Canonical"][i] = canonical


In [ ]:
scopeNames = nameDF['ID']
scopeSMILES = smilesDF['SMILES']
dfNames = df_MASTER['Compound_Name']
yields = smilesDF['Yield']
#print(df_MASTER)

In [ ]:
def insertIntoDataframe(df1 ,  connectingStr ,connectingStr2 ,  df2 , inputStr , indexInterest  ):
    #First create an empty column of zeros at the index of interest and call it inputStr 
    df1.insert(indexInterest, inputStr, 0)
    
    col1 = list(df1[connectingStr])

    col2 = list(df2[connectingStr2])
    insertCol = list(df2[inputStr])
    

    for i , str in enumerate (col1):
        if str in col2:
            index = col2.index(str)
            insertVal = insertCol[index]
            #df1[inputStr][i] = insertVal
            df1.loc[i, inputStr] = insertVal
    return df1

In [ ]:
display(df_MASTER)

In [ ]:
dfNew = insertIntoDataframe(df_MASTER ,  "Compound_Name" ,"ID" ,  nameDF , "SMILES" , 0  )

In [ ]:
display(dfNew)


In [ ]:
dfNew = insertIntoDataframe(dfNew ,  "SMILES" ,"Canonical" ,  smilesDF , "Yield" , 2  )

In [ ]:
display(dfNew)


In [ ]:
import json
alkenePool = "/Users/danielruiz/Downloads/Alkenes/alkenePool/substratePool.json"
with open(alkenePool, 'r') as f:
    alkeneDict = json.load(f)
canonicalSMILES = list(alkeneDict.keys())

In [ ]:
def convertCanonical(str):
    from rdkit import Chem
    mol = Chem.MolFromSmiles(str)
    canonical = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    return canonical

In [ ]:
smiles1 = list(dfNew["SMILES"])

dfNew.insert(3, "ChemistryType", 0)
for i , smile in enumerate(smiles1):
    canonical = convertCanonical(smile)
    if canonical in canonicalSMILES:
        chemType = int(alkeneDict[canonical])
        dfNew["ChemistryType"][i] = chemType
        
    


In [ ]:
dfNew.to_csv("mCPBAALKS_MASTDF.csv", index=False)

In [ ]:
display(dfNew)
